In [1]:
import pandas as pd
import matplotlib.pyplot as plt

### 1. Lectura de los datos con python. Ficheros de navegación y de conversión

In [2]:
data_location_navegacion = 'navegacion.csv'
data_location_conversiones = 'conversiones.csv'
df_navegacion = pd.read_csv(data_location_navegacion)
df_conversiones = pd.read_csv(data_location_conversiones)

### 2. Separación de los datos en columnas y concatenarlo a navegacion
Obtener por cada línea de navegación: campaña, adgoup, advertisement y site link que se obtiene de la columna URL.

SE HAN OBTENIDO TODOS LOS CAMPOS DE LA URL EN UN DATAFRAME

In [6]:
def separate_data(df_navegacion):
    url_list_withoutnan = eliminate_nanvalues_from_url(df_navegacion)
    columns_list = ['URL_base&landing', 'glid', 'iduser', 'uuid', 'camp', 'adg', 'device','sl', 'adv', 'rec', 'extra']
    df_url = create_url_dataframe(columns_list)
    df_url_completed = get_url_data(df_url, url_list_withoutnan)
    df_url_completed.to_csv("df_url_completed.csv", index = False)
    df_navegacion_url_separated = pd.concat([df_navegacion, df_url_completed], axis = 1)
    df_navegacion_url_separated.to_csv("df_navegacion_url_separated.csv", index = False)
    
    return df_navegacion_url_separated
    
df_navegacion_url_separated = separate_data(df_navegacion)


#### 2.1. Eliminar valores NAN de la columna de url

In [3]:
def eliminate_nanvalues_from_url(df_navegacion):
    url_list = df_navegacion["url_landing"]
    url_list_withoutnan = [x for x in url_list if x != 'NaN']
    df_navegacion["url_landing"] = url_list_withoutnan
    
    return url_list_withoutnan

#### 2.2. Crear un dataframe para dividir los campos de cada url

In [4]:
def create_url_dataframe(columns_list):
    df_url = pd.DataFrame(columns = columns_list)
    return df_url

#### 2.3. Dividir la url e introducir los valores en cada uno de sus campos en la columna correspondiente

In [5]:
def get_url_data(df_url, url_list_withoutnan):
    i = 0
    for url in url_list_withoutnan:
        if (url != "") and (isinstance(url, str)):
            fila = []
            
            #1. Se parte la url por el elemento '&'
            splitted_url = url.split('&')

            # 2. Se toma el primer y el segundo elemento de splitted_url. 
            #La primera parte contiene url_base y landing. La segunda parte gclid
            splitted_url_landing_glid = splitted_url[0].split('gclid=')

            # 3. Se parte de nuevo el primer elemento de splitted_url_landing_glid. 
            # Se parte en url_landing y en glid
            url_landing = splitted_url_landing_glid[0]
            if(len(splitted_url_landing_glid) >= 2):
                glid = splitted_url_landing_glid[1]

            fila = splitted_url
            fila[0] = glid
            fila.insert(0, url_landing)

            if len(fila) < 11:
                fila.append(" ")

            df_url.loc[i] = fila
        i = i + 1
    return df_url

### 3. Identificar usuarios repetidos

In [8]:
df_navegacion_url_separated = pd.read_csv('df_navegacion_url_separated.csv')
usuarios_unicos = df_navegacion_url_separated['id_user'].unique()
len(usuarios_unicos)

5798

In [ ]:
df_navegacion_url_separated[df_navegacion_url_separated.id_user.isnull()]


In [ ]:
navegacion_url_separated_duplicated = df_navegacion_url_separated['id_user'].duplicated()
print(navegacion_url_separated_duplicated)

In [ ]:
usuarios_repetidos = 0
for element in navegacion_url_separated_duplicated:
    if element == True:
        usuarios_repetidos = usuarios_repetidos + 1

print(usuarios_repetidos)

#### 3.1. Orden de los usuarios por la columna ts para quedarnos con los primeros usuarios no repetidos

In [7]:
df_navegacion_url_separated.sort_values(['ts', 'id_user'], ascending=[True, False])

,ts,uuid,id_user,gclid,user_recurrent,url_landing,URL_base&landing,glid,iduser,uuid,camp,adg,device,sl,adv,rec,extra
16,2021-09-06 0:10:32.157,c31cef27-700f-4e75-a5e2-2f10427b8b9b,ad983b54-d48c-4aa8-9241-b5c49d83428c,Cj0KCQjw1dGJBhD4ARIsANb6OdnCz3gI3tgGzbLgHXP-br...,True,https://www.metropolis.com/es/gclid=Cj0KCQjw1d...,https://www.metropolis.com/es/,Cj0KCQjw1dGJBhD4ARIsANb6OdkQSu1pDM8Ak-pZzOEzL1...,idUser=666c9866-b5a4-4f4c-9985-a6a8cbf4e680,uuid=d84e9cdd-08b5-405d-97c7-eb78ad187a5b,camp=732187328,adg=46724581628,device=c,sl=,adv=533655604703,rec=true,
17,2021-09-06 0:11:10.616,d84e9cdd-08b5-405d-97c7-eb78ad187a5b,666c9866-b5a4-4f4c-9985-a6a8cbf4e680,Cj0KCQjw1dGJBhD4ARIsANb6OdkQSu1pDM8Ak-pZzOEzL1...,True,https://www.metropolis.com/es/cea/gclid=Cj0KCQ...,https://www.metropolis.com/es/cea/,Cj0KCQjw1dGJBhD4ARIsANb6OdnNoir5uib6uzhGF9uQ7D...,idUser=3f146eaf-172f-45c2-8d23-fba1446eb653,uuid=a6456565-3f3e-4228-ae6e-d186e17a0eda,camp=1648646316,adg=58613330770,device=m,sl=55827747411,adv=494939238438,rec=true,
18,2021-09-06 0:11:10.739,a6456565-3f3e-4228-ae6e-d186e17a0eda,3f146eaf-172f-45c2-8d23-fba1446eb653,Cj0KCQjw1dGJBhD4ARIsANb6OdnNoir5uib6uzhGF9uQ7D...,True,https://www.metropolis.com/es/gclid=EAIaIQobCh...,https://www.metropolis.com/es/,EAIaIQobChMIwJWn_ofp8gIVBuR3CcomY9QfTEAAYASAAE...,idUser=e55feb88-da64-4eca-abb5-57385ad01853,uuid=579d80ab-9632-435d-8158-a43a2a3b10f6,camp=732187328,adg=46724581628,device=c,sl=,adv=533655604703,rec=false,
19,2021-09-06 0:12:9.065,579d80ab-9632-435d-8158-a43a2a3b10f6,e55feb88-da64-4eca-abb5-57385ad01853,EAIaIQobChMIwJWn_ofp8gIVBuR3Ch1Y9QfTEAAYASAAEg...,False,https://www.metropolis.com/es/tria-hibrido/gcl...,https://www.metropolis.com/es/tria-hibrido/,Cj0KCQjw1dGJBhD4ARIsANb6OdnfkWZhSxlOmTpAZFOC7e...,idUser=9ca6ce29-90d1-4cab-b017-4281a1699427,uuid=2f2757bd-ac81-41b0-9188-aa7e320e3ea0,camp=1725352245,adg=73019156532,device=t,sl=,adv=481025151991,rec=false,
20,2021-09-06 0:13:4.787,2f2757bd-ac81-41b0-9188-aa7e320e3ea0,9ca6ce29-90d1-4cab-b017-4281a1699427,Cj0KCQjw1dGJBhD4ARIsANb6OdnfkWZhSxlOmTpAZFOC7e...,False,https://www.metropolis.com/es/tria/gclid=Cj0KC...,https://www.metropolis.com/es/tria/,Cj0KCQjw1dGJBhD4ARIsANb6OdkGinPuBStihlgkhxJFsY...,idUser=ca21f8f5-b7ef-42e0-96dc-759d413e36ca,uuid=7d557b7e-802d-430c-8b3b-a99750d3afd6,camp=1648648995,adg=62589383945,device=m,sl=,adv=481025151796,rec=false,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7263,2021-09-07 0:2:46.493,5a54eec3-c3e6-41f7-9eb1-d87620d9ab0d,d6ac974a-8fc4-4aeb-87a0-dcbe059339ae,EAIaIQobChMIwPu5t4qs3AIVAQAAAB0BAAAAEAAYACAAEg...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7264,2021-09-07 0:2:50.911,76235cc0-53a7-4460-b2df-bdd2e3a025e2,3f92c7a2-794e-4203-83ba-951fd5fef039,EAIaIQobChMI6I7x4sfr8gIVFODtCh04WAoGEAAYASAAEg...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7266,2021-09-07 0:3:24.716,c77ca194-dc82-4255-a2b4-aed45cec7429,b312d553-b16f-4d81-8d21-91b5fd9549ab,Cj0KCQjw-NaJBhDsARIsAAja6dNkpp0g2prrpjpOa0tZw9...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7267,2021-09-07 0:3:31.914,76235cc0-53a7-4460-b2df-bdd2e3a025e2,3f92c7a2-794e-4203-83ba-951fd5fef039,EAIaIQobChMI6I7x4sfr8gIVFODtCh04WAoGEAAYASAAEg...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 3.2. Eliminar usuarios repetidos

Como ahora tenemos todos los datos ordenados por ts y por usuario y drop_duplicates se queda con el primer usuario de todos los repetidos, nos aseguramos quedarnos con el registro del usuario que accedió antes.

In [8]:
df_navegacion_urlseparated_noduplicates = df_navegacion_url_separated.drop_duplicates(subset=['id_user'])
df_navegacion_urlseparated_noduplicates.to_csv("df_navegacion_urlseparated_noduplicates.csv", index = False)

### 4. Unir tablas
Para ello hay que buscar id_user en conversiones. Si se encuentra se añade un 1 en una nueva columna llamada conversiones y 0 en caso contrario. Además se toman los valores de id_lead, lead_type y result_list

In [13]:
def añadir_columnas_conversion(df_navegacion_urlseparated_noduplicates, df_conversiones_withdates):
    
    conversion_list = []
    id_lead_list = []
    lead_type_list = []
    result_list = []
    hour_list = []
    day_list = []
    month_list = []
    year_list = []
    
    users_conversiones_list = df_conversiones_withdates['id_user'].tolist()
    print(users_conversiones_list)
    
    # Por cada fila de navegacion
    for row in df_navegacion_without_reps_url_separated.itertuples():
        id_user = row.id_user
        
        str_match = [s for s in users_conversiones_list if s.__contains__(id_user)]
        
        # Si hay match
        if str_match != []:
            print(str_match[0])
            
            # Nos recorremos la tabla de conversiones en busca de la fila que contiene el id_user con el que ha hecho match
            for row_conversiones in df_conversiones_withdates.itertuples():
                print(row_conversiones.id_user)
                if str_match[0] == str(row_conversiones.id_user):
                    element_conversion = 1
                    element_id_lead = row_conversiones.id_lead
                    element_lead_type = row_conversiones.lead_type
                    element_list_append = row_conversiones.result
                    element_list_hour = row_conversiones.hour
                    element_list_day = row_conversiones.day
                    element_list_month = row_conversiones.month
                    element_list_year = row_conversiones.year
                    
                    conversion_list.append(element_conversion)               
                    id_lead_list.append(element_id_lead)
                    lead_type_list.append(element_lead_type)
                    result_list.append(element_list_append)
                    hour_list.append(element_list_hour)
                    day_list.append(element_list_day)
                    month_list.append(element_list_month)
                    year_list.append(element_list_year)
                    
        else:
            element_conversion = 0
            element_id_lead = '-'
            element_lead_type = '-'
            element_list_append = '-'
            element_list_hour = '-'
            element_list_day = '-'
            element_list_month = '-'
            element_list_year = '-'
            
            conversion_list.append(element_conversion)
            id_lead_list.append(element_id_lead)
            lead_type_list.append(element_lead_type)
            result_list.append(element_list_append)
            hour_list.append(element_list_hour)
            day_list.append(element_list_day)
            month_list.append(element_list_month)
            year_list.append(element_list_year)

    
    df_navegacion_without_reps_url_separated['conversiones'] = pd.Series(conversion_list)
    df_navegacion_without_reps_url_separated['id_lead'] = pd.Series(id_lead_list)
    df_navegacion_without_reps_url_separated['lead_type'] = pd.Series(lead_type_list)
    df_navegacion_without_reps_url_separated['result'] = pd.Series(result_list)
    df_navegacion_without_reps_url_separated['hour_conversion'] = pd.Series(hour_list)
    df_navegacion_without_reps_url_separated['day_conversion'] = pd.Series(day_list)
    df_navegacion_without_reps_url_separated['month_conversion'] = pd.Series(month_list)
    df_navegacion_without_reps_url_separated['year_conversion'] = pd.Series(year_list)
    
    
    df_navegacion_without_reps_url_separated.to_csv("df_navegacion_urlseparated_noduplicates_conversiones_concatenated.csv", index = False)

df_navegacion_without_reps_url_separated = pd.read_csv("df_navegacion_urlseparated_noduplicates.csv")  
df_conversiones_withdates = pd.read_csv("df_conversiones_withdates.csv")
añadir_columnas_conversion(df_navegacion_without_reps_url_separated, df_conversiones_withdates)

[' cefd8f05-8dd2-4e82-869f-1ddc57a9a4c3', ' None', ' None', ' None', ' None', ' None', ' None', ' None', ' None', ' None', ' e942cf56-2aa1-462b-b561-dd47b332e880', ' None', ' None', ' None', ' 765f65aa-dea5-49e2-a0b8-656aa669b2fb', ' None', ' 193f16ad-6dc0-401f-9593-06bfafb32264', ' fbee18ca-6970-4015-8892-814e664b3dab', ' 2f59a840-26c4-4a41-b1b0-0c3cd3b669bf', ' 3f336d07-2513-401a-984a-f258c032d686', ' 1d792022-78e4-4af9-9ea5-741117389e66', ' 2f616f73-6d38-4041-95e9-911b90fbaee2', ' 215134e8-3170-43dc-8f08-f06438129a9c', ' 8fb314e8-b0fe-4e17-b398-c0c5d2a988d0', ' ebde412f-7daf-4349-b888-fbf1d9deb04b', ' 59257b75-5389-49e4-b1aa-5997ecfdc7f9', ' 3f336d07-2513-401a-984a-f258c032d686', ' fff88164-ce92-486c-a4d8-41c7b9779f24']
 59257b75-5389-49e4-b1aa-5997ecfdc7f9
 cefd8f05-8dd2-4e82-869f-1ddc57a9a4c3
 None
 None
 None
 None
 None
 None
 None
 None
 None
 e942cf56-2aa1-462b-b561-dd47b332e880
 None
 None
 None
 765f65aa-dea5-49e2-a0b8-656aa669b2fb
 None
 193f16ad-6dc0-401f-9593-06bfafb32264

### 5. Separar la columna ts en las columnas de hora, día, mes y año

#### MAIN FUNCTION

In [ ]:
def create_customers_conversiones_withconversiones_withtime(dataframe):
    df_dates = separate_ts(dataframe)
    
    dataframe = dataframe.drop(['ts'], axis = 1)
    df_navegacion_without_reps_url_separated_conversiones_concatenated_withtime = pd.concat([df_dates, dataframe], axis = 1)
    df_navegacion_without_reps_url_separated_conversiones_concatenated_withtime.to_csv('df_navegacion_urlseparated_noduplicates_conversiones_concatenated_withtime.csv', index = False)
    
df_navegacion_without_reps_url_separated_conversiones_concatenated = pd.read_csv("df_navegacion_urlseparated_noduplicates_conversiones_concatenated.csv")  
create_customers_conversiones_withconversiones_withtime(df_navegacion_without_reps_url_separated_conversiones_concatenated)

In [10]:
def separate_ts(dataframe):
    ts = dataframe["ts"]
    # Guardar en un Dataframe la fecha (Día, mes, año)
    columns_list = ["day", "month", "year", "hour"]
    df_dates = pd.DataFrame(columns = columns_list)
    print(df_dates)

    i = 1
    for t in ts:
        df_dates_row = []
        splitted_t = t.split(' ')
        fecha = splitted_t[0]
        hour = splitted_t[1]
        splitted_fecha = fecha.split('-')
        day = splitted_fecha[2]
        month = splitted_fecha[1]
        year = splitted_fecha[0]

        df_dates_row.append(day)
        df_dates_row.append(month)
        df_dates_row.append(year)
        df_dates_row.append(hour)

        print(df_dates_row)

        df_dates.loc[len(df_dates.index)] = df_dates_row

        i = i + 1

    return df_dates

Separación de la fecha en conversiones.csv

In [11]:
# Guardar en un Dataframe la fecha (Día, mes, año)
columns_list = ["day", "month", "year"]
df_dates = pd.DataFrame(columns = columns_list)
dates = df_conversiones["date"]
print(df_dates)

for date in dates:
    splitted_date = date.split('/')
    df_dates.loc[len(df_dates.index)] = splitted_date
    
df_conversiones_withdates = pd.concat([df_dates, df_conversiones], axis = 1)
df_conversiones_withdates.to_csv('df_conversiones_withdates.csv', index = False)

Empty DataFrame
Columns: [day, month, year]
Index: []
